In [116]:
import pandas as pd
import json
from glob import glob
import pathlib as pl
from types import SimpleNamespace
import seaborn as sns
import matplotlib.pyplot as plt
from collections import OrderedDict

pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.10f' % x)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
sns.set_context('poster')

dir_root = pl.Path('').absolute()
dir_calc_mw = dir_root.joinpath('calcs_mw_zora')
dir_calc_gto = dir_root.joinpath('calcs_gto_zora')

molecules = ['He', 'Ne', 'Ar', 'Kr', 'LiH', 'H2O']
cs = ['14', '137', '137036']

takes = OrderedDict({0: 'Take 1', 1: 'Take 2', 2: 'Take 3'})
kins = OrderedDict({0: 'std', 1: 'sym', 2: 'sqr'})

In [131]:
def MWEnergy(f):
    with open(f) as out:
        job = json.loads(out.read())
    try:
        if job['output']['success']:
            return float(job['output']['properties']['scf_energy']['E_tot'])
        else:
            return None
    except KeyError:
        return None

def GTOEnergy(f):
    with open(f) as out:
        lines = out.readlines()
    for line in lines:
        if line.strip().startswith('FINAL SINGLE POINT ENERGY'):
            return float(line.split()[4])
        
def numberOfSCFIterations(f):
    with open(f) as out:
        job = json.loads(out.read())
    try:
        if job['output']['success']:
            return str(len(job['output']['scf_calculation']['scf_solver']['cycles']))
        else:
            return None
    except KeyError:
        return None

In [132]:
table = []
for mol in molecules:
    for c in cs:
        f_gto = dir_calc_gto.joinpath(f'{mol}_{c}_.out')
        e_gto = GTOEnergy(f_gto)
        for kin, kinlabel in kins.items():
            for take, takelabel in takes.items():
                f_mw = dir_calc_mw.joinpath(f'{mol}_{kin}_{take}_{c}_.json')
                e_mw = MWEnergy(f_mw)
                niter = numberOfSCFIterations(f_mw)
        
                table.append((mol, c, takelabel, kinlabel, niter, e_gto, e_mw))

In [135]:
df = pd.DataFrame(table, columns=['Molecule', 'c', 'Take', 'Kinetic', 'nIter', 'def2-QZVPP', 'MW6'])
df['Error'] = df['def2-QZVPP'] - df.MW6

df = df.sort_values(by=['c', 'Molecule', 'Take', 'Kinetic'], ascending=False).reset_index(drop=True)


In [136]:
for mol, g in df.groupby('Molecule'):
    print('==========================================================================================')
    i = 0
    for take, sub in g.groupby('Take'):
        i += 1
        if i > 1:
            print('------------------------------------------------------------------------------------------')
            print('\n'.join(sub.to_string(index=False, header=True).split('\n')[1:]))
        else:
            print(sub.to_string(index=False, header=True))

Molecule      c   Take Kinetic nIter      def2-QZVPP             MW6        Error
      Ar     14 Take 1     sym  None -680.1889107096             NaN          NaN
      Ar     14 Take 1     std  None -680.1889107096             NaN          NaN
      Ar     14 Take 1     sqr  None -680.1889107096             NaN          NaN
      Ar 137036 Take 1     sym     2 -312.6874360661 -312.6875153796 0.0000793136
      Ar 137036 Take 1     std     2 -312.6874360661 -312.6875153796 0.0000793136
      Ar 137036 Take 1     sqr     2 -312.6874360661 -312.6875153796 0.0000793136
      Ar    137 Take 1     sym     8 -315.3827353973 -315.3974746180 0.0147392207
      Ar    137 Take 1     std     8 -315.3827353973 -315.3974746180 0.0147392207
      Ar    137 Take 1     sqr     8 -315.3827353973 -315.3974746357 0.0147392384
------------------------------------------------------------------------------------------
      Ar     14 Take 2     sym  None -680.1889107096             NaN          NaN
      A

In [23]:
import math
mass_e = 0.000548
mass_p = 1.007276
mass_n = 1.008665

ratio_hydrogen = mass_p / mass_e
ratio_carbon = (6*mass_p + 6*mass_n) / mass_e

radius_nuc = 2.5
radius_H = radius_nuc / math.sqrt(ratio_hydrogen)
radius_C = radius_nuc / math.sqrt(ratio_carbon)

In [24]:
radius_C

0.01682735100791286